<a href="https://colab.research.google.com/github/geeta-gwalior/sql-query-generator-geminiai/blob/main/Gemini_with_Rag.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# RAG application built on gemini

# New section

In [ ]:
from langchain_community.document_loaders import PyPDFLoader

loader = PyPDFLoader("edufund.pdf")
data = loader.load()
data

[Document(metadata={'source': 'edufund.pdf', 'page': 0}, page_content='WHAT ARE MUTUAL FUNDS?\nA mutual fund is a pool of money managed by a professional Fund Manager.\nIt is a trust that collects money from a number of investors who share a common investment objective and invests the same in equities, bonds, money market instruments and/or other securities. And the income / gains generated from this collective investment is distributed proportionately amongst the investors after deducting applicable expenses and levies, by calculating a scheme’s “Net Asset Value” or NAV. Simply put, the money pooled in by a large number of investors is what makes up a Mutual Fund.\nHere’s a simple way to understand the concept of a Mutual Fund Unit.\u2028Let’s say that there is a box of 12 chocolates costing ₹40. Four friends decide to buy the same, but they have only ₹10 each and the shopkeeper only sells by the box. So the friends then decide to pool in ₹10 each and buy the box of 12 chocolates. Now

In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

# split data
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=1000)
docs = text_splitter.split_documents(data)


print("Total number of documents: ",len(docs))
for chunk in docs:
    print(chunk)
print(docs[0].page_content)

Total number of documents:  72
page_content='WHAT ARE MUTUAL FUNDS?
A mutual fund is a pool of money managed by a professional Fund Manager.
It is a trust that collects money from a number of investors who share a common investment objective and invests the same in equities, bonds, money market instruments and/or other securities. And the income / gains generated from this collective investment is distributed proportionately amongst the investors after deducting applicable expenses and levies, by calculating a scheme’s “Net Asset Value” or NAV. Simply put, the money pooled in by a large number of investors is what makes up a Mutual Fund.' metadata={'source': 'edufund.pdf', 'page': 0}
page_content='Here’s a simple way to understand the concept of a Mutual Fund Unit. Let’s say that there is a box of 12 chocolates costing ₹40. Four friends decide to buy the same, but they have only ₹10 each and the shopkeeper only sells by the box. So the friends then decide to pool in ₹10 each and buy th

In [ ]:
from langchain_chroma import Chroma
from langchain_google_genai import GoogleGenerativeAIEmbeddings

from dotenv import load_dotenv
from google.colab import userdata



embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001",google_api_key=userdata.get('gemini_api'))
vector = embeddings.embed_query("hello, world!")
len(vector)
vector[0]


0.05168594419956207

In [ ]:
vectorstore = Chroma.from_documents(documents=docs, embedding=embeddings)

In [ ]:
retriever = vectorstore.as_retriever(search_type="similarity", search_kwargs={"k": 5})

retrieved_docs = retriever.invoke("What is Mutual funds")
len(retrieved_docs)
print(retrieved_docs[0].page_content)

WHAT ARE MUTUAL FUNDS?
A mutual fund is a pool of money managed by a professional Fund Manager.
It is a trust that collects money from a number of investors who share a common investment objective and invests the same in equities, bonds, money market instruments and/or other securities. And the income / gains generated from this collective investment is distributed proportionately amongst the investors after deducting applicable expenses and levies, by calculating a scheme’s “Net Asset Value” or NAV. Simply put, the money pooled in by a large number of investors is what makes up a Mutual Fund.


In [ ]:
from langchain_google_genai import ChatGoogleGenerativeAI

llm = ChatGoogleGenerativeAI(model="gemini-1.5-flash",temperature=0.3)

In [ ]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate

system_prompt = (
    "You are an assistant for question-answering tasks. "
    "Use the following pieces of retrieved context to answer "
    "the question. If you don't know the answer, say that you "
    "don't know. Use three sentences maximum and keep the "
    "answer concise."
    "\n\n"
    "{context}"
)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)

In [ ]:
question_answer_chain = create_stuff_documents_chain(llm, prompt)
rag_chain = create_retrieval_chain(retriever, question_answer_chain)

In [ ]:
response=rag_chain.invoke({"input":"What is Mutual funds"})
print(response)

Streaming output truncated to the last 5000 lines.

Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/grpc/_plugin_wrapping.py", line 105, in __call__
    self._metadata_plugin(
  File "/usr/local/lib/python3.10/dist-packages/google/auth/transport/grpc.py", line 95, in __call__
    callback(self._get_authorization_headers(context), None)
  File "/usr/local/lib/python3.10/dist-packages/google/auth/transport/grpc.py", line 81, in _get_authorization_headers
    self._credentials.before_request(
  File "/usr/local/lib/python3.10/dist-packages/google/auth/credentials.py", line 228, in before_request
    self._blocking_refresh(request)
  File "/usr/local/lib/python3.10/dist-packages/google/auth/credentials.py", line 191, in _blocking_refresh
    self.refresh(request)
  File "/usr/local/lib/python3.10/dist-packages/google/auth/compute_engine/credentials.py", line 134, in refresh
    raise new_exc from caught_exc
google.auth.exceptions.RefreshError: ("Failed to

RetryError: Timeout of 600.0s exceeded, last exception: 503 Getting metadata from plugin failed with error: ("Failed to retrieve http://metadata.google.internal/computeMetadata/v1/instance/service-accounts/default/?recursive=true from the Google Compute Engine metadata service. Status: 404 Response:\nb''", <google.auth.transport.requests._Response object at 0x7988e2ee9060>)